In [ ]:
import os
import sys
import torch
import types
import importlib
import pickle

# Add the paths (you already have this)
sys.path.insert(0, '..')
sys.path.insert(0, '../..')
sys.path.insert(0, '../../..')
sys.path.insert(0, '../../../..')
sys.path.insert(0, '../../../../..')
sys.path.insert(0, '../../../../../..')

from probabilistic_suffix_prediction_dev.src.model.dropout_uncertainty_enc_dec_LSTM.dropout_uncertainty_model import DropoutUncertaintyEncoderDecoderLSTM

# Step 2: Import the actual module
from probabilistic_suffix_prediction_dev.src.event_log_loader import new_event_log_loader

# Step 3: Create a dummy package module for `event_log_loader`
event_log_loader_pkg = types.ModuleType("event_log_loader")
sys.modules["event_log_loader"] = event_log_loader_pkg

# Step 4: Register the real module under the expected name
sys.modules["event_log_loader.new_event_log_loader"] = new_event_log_loader


In [ ]:
#load model
file_path_model = '../../../../probabilistic_suffix_prediction_dev/src/notebooks/training_variational_dropout/Helpdesk/Helpdesk_full_grad_norm_new_4layer.pkl'

# WITHOUT DROPOUT:

model = DropoutUncertaintyEncoderDecoderLSTM.load(file_path_model, dropout=0.0)

# Load the dataset
file_path_data_set = '../../../../probabilistic_suffix_prediction_dev/encoded_data/helpdesk_all_5_val.pkl'
helpdesk_test_dataset = torch.load(file_path_data_set, weights_only=False)

In [ ]:
import probabilistic_suffix_prediction_dev.src.evaluation.probabilistic_evaluation as pe
importlib.reload(pe)
from probabilistic_suffix_prediction_dev.src.evaluation.probabilistic_evaluation import ProbabilisticEvaluation

new_eval = ProbabilisticEvaluation(model, 
                                   helpdesk_test_dataset,
                                   concept_name='Activity',
                                   num_processes=16,
                                   #growing_num_values = [],
                                   growing_num_values = ['case_elapsed_time'],
                                   samples_per_case = 1000,
                                   sample_argmax = False,
                                   use_variance_cat = True,
                                   use_variance_num = True,
                                   all_cat=['Activity', 'Resource'],
                                   all_num=['case_elapsed_time', 'event_elapsed_time'])

In [ ]:
def save_chunk(results, i):
    chunk_number = (i + 1)
    filename = os.path.join(output_dir, f'results_part_{chunk_number:03d}.pkl')
    with open(filename, 'wb') as f:
        pickle.dump(results, f)
    print(f"Saved {len(results)} results to {filename}")

output_dir = '../../../../../data/Helpdesk/conformal_val_without_dropout/'

save_every = 50
results = {}
for i, (case_name, prefix_len, prefix, predicted_suffixes, suffix, mean_prediction) in enumerate(new_eval.evaluate_multi_processing(random_order=True, include_model_states=False)):
    # print(case_name, prefix_len)
    assert((case_name, prefix_len) not in results)
    results[(case_name, prefix_len)] = (prefix, suffix, mean_prediction, predicted_suffixes)
    # print(prefix_len, len(suffix))
    if (i + 1) % save_every == 0:
        save_chunk(results, i)
        results = {}

if len(results):
    save_chunk(results, i)